# Project Part 3

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
input_data_path = 'https://raw.githubusercontent.com/Matthew-Bustamante/CS39AA-Project-Cyberbullying/main/CyberBullying_Comments_Dataset.csv'
df = pd.read_csv(input_data_path)
df.head()

,Text,CB_Label
0,damn there is someones nana up here at beach w...,0
1,no kidding! dick clark was a corpse mechanical...,0
2,i read an article on jobros and thought damn w...,0
3,I got one fucking day of sprinkles and now it'...,0
4,I was already listening to Elliott smith and ...,0


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_metric

In [8]:
MODEL_NAME = "bert-base-cased"
MAX_LENGTH=50 

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, max_length=MAX_LENGTH, output_attentions=False, output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Creating Vocabulary for y(dependent variable).  Note that 0 = non-cyberbullying and 1 = cyberbullying

In [10]:
classes = df.CB_Label.unique().tolist()
class_tok2idx = dict((v, k) for k, v in enumerate(classes))
class_idx2tok = dict((k, v) for k, v in enumerate(classes))
print(class_tok2idx)
print(class_idx2tok)

{0: 0, 1: 1}
{0: 0, 1: 1}


Creating a new column with these labels which will be the y that is used

In [11]:
df['label'] = df['CB_Label'].apply(lambda x: class_tok2idx[x])
df.head()

,Text,CB_Label,label
0,damn there is someones nana up here at beach w...,0,0
1,no kidding! dick clark was a corpse mechanical...,0,0
2,i read an article on jobros and thought damn w...,0,0
3,I got one fucking day of sprinkles and now it'...,0,0
4,I was already listening to Elliott smith and ...,0,0


In [14]:
'''
sequence_0 = "@united I will never fly with you again. Period."
seq0_tokens = tokenizer(sequence_0, return_tensors="pt")
print(f"number of tokens in seq0 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)
'''

number of tokens in seq0 is 14
{'input_ids': tensor([[  101,   137, 10280,   146,  1209,  1309,  4689,  1114,  1128,  1254,
           119, 16477,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


NameError: name 'F' is not defined